In [34]:
%%writefile train.py

import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1)

model.evaluate(x_test, y_test)

Overwriting train.py


In [35]:
%%writefile Dockerfile
FROM tensorflow/tensorflow:2.0.0a0

RUN pip install sagemaker-containers

# Copies the training code inside the container
COPY train.py /opt/ml/code/train.py

# Defines train.py as script entry point
ENV SAGEMAKER_PROGRAM train.py

Overwriting Dockerfile


In [36]:
! docker build -t tf-2.0-with-lib .

Sending build context to Docker daemon  40.96kB
Step 1/5 : FROM tensorflow/tensorflow:2.0.0a0
 ---> 2ebc856b5e27
Step 2/5 : RUN pip install sagemaker-containers
 ---> Using cache
 ---> 4cc501b79a04
Step 3/5 : COPY train.py /opt/ml/code/train.py
 ---> b41a58676845
Step 4/5 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code1
 ---> Running in b3ef3aaeeb13
Removing intermediate container b3ef3aaeeb13
 ---> 66c77f6da09e
Step 5/5 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in 092af81eb191
Removing intermediate container 092af81eb191
 ---> 20c54215d5a3
Successfully built 20c54215d5a3
Successfully tagged tf-2.0-with-lib:latest


In [37]:
import sagemaker
from sagemaker.estimator import Estimator

hyperparameters = {'epochs': 5, 'batch_size': 128, 'learning_rate': 0.01, 'other_para':0.1}

estimator = Estimator(image_name='tf-2.0-with-lib',
                      role=sagemaker.get_execution_role(),
                      hyperparameters=hyperparameters,
                      train_instance_count=1,
                      train_instance_type='local')

estimator.fit()

Creating tmp1q2u1ljd_algo-1-tkilo_1 ... 
Attaching to tmp1q2u1ljd_algo-1-tkilo_12mdone
algo-1-tkilo_1  | 2020-02-19 10:58:17,508 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-tkilo_1  | 2020-02-19 10:58:17,522 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-tkilo_1  | 2020-02-19 10:58:17,534 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-tkilo_1  | 2020-02-19 10:58:17,545 sagemaker-containers INFO     Invoking user script
algo-1-tkilo_1  | 
algo-1-tkilo_1  | Training Env:
algo-1-tkilo_1  | 
algo-1-tkilo_1  | {
algo-1-tkilo_1  |     "module_dir": "/opt/ml/code", 
algo-1-tkilo_1  |     "channel_input_dirs": {}, 
algo-1-tkilo_1  |     "resource_config": {
algo-1-tkilo_1  |         "hosts": [
algo-1-tkilo_1  |             "algo-1-tkilo"
algo-1-tkilo_1  |         ], 
algo-1-tkilo_1  |         "current_host": "algo-1-tkilo"
algo-1-tkilo_1  |     }, 
algo-1-tkilo_1  |     "num_cpus":

In [38]:
%%writefile Dockerfile
FROM tensorflow/tensorflow:2.0.0a0

# RUN pip install sagemaker-containers

# Copies the training code inside the container
COPY train.py /opt/ml/code/train.py

# Defines train.py as script entry point
ENV SAGEMAKER_PROGRAM train.py

Overwriting Dockerfile


In [39]:
! docker build -t tf-2.0-without-lib .

Sending build context to Docker daemon  41.47kB
Step 1/3 : FROM tensorflow/tensorflow:2.0.0a0
 ---> 2ebc856b5e27
Step 2/3 : COPY train.py /opt/ml/code/train.py
 ---> 33b8eb8101cc
Step 3/3 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in 66d88c5f8a02
Removing intermediate container 66d88c5f8a02
 ---> d39487d0d8d2
Successfully built d39487d0d8d2
Successfully tagged tf-2.0-without-lib:latest


In [40]:
import sagemaker
from sagemaker.estimator import Estimator

hyperparameters = {'epochs': 5, 'batch_size': 128, 'learning_rate': 0.01, 'other_para':0.1}

estimator = Estimator(image_name='tf-2.0-without-lib',
                      role=sagemaker.get_execution_role(),
                      hyperparameters=hyperparameters,
                      train_instance_count=1,
                      train_instance_type='local')

estimator.fit()

Creating tmpsc_hp6h9_algo-1-2cplg_1 ... 
ting tmpsc_hp6h9_algo-1-2cplg_1 ... error
ERROR: for tmpsc_hp6h9_algo-1-2cplg_1  Cannot start service algo-1-2cplg: OCI runtime create failed: container_linux.go:345: starting container process caused "exec: \"train\": executable file not found in $PATH": unknown

ERROR: for algo-1-2cplg  Cannot start service algo-1-2cplg: OCI runtime create failed: container_linux.go:345: starting container process caused "exec: \"train\": executable file not found in $PATH": unknown
Encountered errors while bringing up the project.


RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpsc_hp6h9/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

In [13]:
! cat /tmp/tmpdn05x743/docker-compose.yaml

networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-dw4a3:
    command: train
    environment:
    - AWS_REGION=us-east-2
    - TRAINING_JOB_NAME=tf-2.0-without-lib-2020-02-19-08-13-15-912
    image: tf-2.0-without-lib
    networks:
      sagemaker-local:
        aliases:
        - algo-1-dw4a3
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmpdn05x743/algo-1-dw4a3/output:/opt/ml/output
    - /tmp/tmpdn05x743/algo-1-dw4a3/output/data:/opt/ml/output/data
    - /tmp/tmpdn05x743/algo-1-dw4a3/input:/opt/ml/input
    - /tmp/tmpdn05x743/model:/opt/ml/model
version: '2.3'


In [24]:
%%writefile train1.py
#!/usr/bin/env python

import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1)

model.evaluate(x_test, y_test)

Writing train1.py


In [28]:
%%writefile Dockerfile
FROM tensorflow/tensorflow:2.0.0a0

# RUN pip install sagemaker-containers

# Copies the training code inside the container
# train作为脚本将被之行
COPY train1.py /opt/ml/code/train

RUN chmod 777 /opt/ml/code/train

# 将此目录添加到PATH中，因为sagemaker会之行docker run CONTAINER_ID train, train作为命令需要添加到PATH中
ENV PATH="/opt/ml/code:${PATH}"

# 指定工作目录
WORKDIR /opt/ml/code

# Defines train.py as script entry point
# 如果没有sagemaker-containers，这个环境变量没有用ENV SAGEMAKER_PROGRAM train.py

Overwriting Dockerfile


In [29]:
# 
! docker build -t tf-2.0-without-lib-fixed .

Sending build context to Docker daemon  40.96kB
Step 1/5 : FROM tensorflow/tensorflow:2.0.0a0
 ---> 2ebc856b5e27
Step 2/5 : COPY train1.py /opt/ml/code/train
 ---> Using cache
 ---> 77a7d58ad757
Step 3/5 : RUN chmod 777 /opt/ml/code/train
 ---> Using cache
 ---> c4c6a9316d0b
Step 4/5 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 5afe63d43b22
Step 5/5 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 1f9e7dfef833
Successfully built 1f9e7dfef833
Successfully tagged tf-2.0-without-lib-fixed:latest


In [30]:
import sagemaker
from sagemaker.estimator import Estimator

hyperparameters = {'epochs': 5, 'batch_size': 128, 'learning_rate': 0.01, 'other_para':0.1}

estimator = Estimator(image_name='tf-2.0-without-lib-fixed',
                      role=sagemaker.get_execution_role(),
                      hyperparameters=hyperparameters,
                      train_instance_count=1,
                      train_instance_type='local')

estimator.fit()

Creating tmp20oye3fi_algo-1-3vg62_1 ... 
Attaching to tmp20oye3fi_algo-1-3vg62_12mdone
algo-1-3vg62_1  | Downloading data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
11501568/11490434 [==============================] - 0s 0us/step
algo-1-3vg62_1  | 2020-02-19 10:17:19.418049: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
algo-1-3vg62_1  | 2020-02-19 10:17:19.439793: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2500000000 Hz
algo-1-3vg62_1  | 2020-02-19 10:17:19.440077: I tensorflow/compiler/xla/service/service.cc:162] XLA service 0x4c1ca90 executing computations on platform Host. Devices:
algo-1-3vg62_1  | 2020-02-19 10:17:19.440105: I tensorflow/compiler/xla/service/service.cc:169]   StreamExecutor device (0): <undefined>, <undefined>
10000/10000 [==============================] - 0s 30us/sample - loss: 0.1417 - accuracy